DISCIPLINA: Gestão do Conhecimento	 PROFESSOR(A): Alex Salgado
PERÍODO: 7o. TURNO: noite AVALIAÇÃO:

ALUNO(A): _

GRAU: VISTO DO PROFESSOR:


# Questão 1 - valor (1,0)


*** Crie um repositório no Github e adicione este arquivo no mesmo


1.1 - Você deve escolher uma base de dados aberta que seja possível treinar um modelo de aprendizagem de máquina Supervisionado de Classificação (assim como o exemplo da Iris e Cancer). A partir desta base de dados, você vai treinar o modelo, fazer previsões e calcular a acurácia do mesmo.
Obs.: Não utilizar nenhuma base de dados usada em aula ou nos trabalhos.


In [0]:
https://github.com/Izalena/machine_learning_av2.git

1.2 - Após tratada, criar uma pasta no seu GoogleDrive e disponibilizar o link do arquivo de sua base (csv, xlsx, etc) para ser importado no seu código.

In [0]:
https://drive.google.com/file/d/1E_XTl2VaaOZqBAW-AQGF9K3J1Hicqqvo/view?usp=sharing / https://drive.google.com/file/d/1FlmF81gWhWi7H_NO9c9-youTvmmSPRZR/view?usp=sharing

## Criar um código usando o Jupyter Notebook e responder (através de código) às seguintes questões:


# Questão 2 - valor (1,0)

2.1 - Importar os modulos python para machine learn e carregar o arquivo 

** Sugestão se for do tipo xlsx, usar o read_excel do pandas
import pandas as pd
dt = pd.read_excel("meuarquivo.xlsx")


In [1]:
import numpy as np
import math
import nltk
import pickle
import os

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('data/train.csv', encoding='latin-1')
test = pd.read_csv('data/test.csv', encoding='latin-1')

train.head(10)



ModuleNotFoundError: No module named 'numpy'

2.3 - De que se trata esse banco de dados? e que tipo de previsão pode ser feito com ele?


In [0]:
São publicações feitas no twitter, os "twits". O objetivo é fazer uma análise de sentimentos.

# Questão 3 - valor (0,5)


Utilizando as terminologias de Machine Learning(features e observações):



3.1 - Quantas "features" têm nessa base de dados?

3.2 - Quantas observações têm nessa base de dados?


# Questão 4 - valor (1,0)


4.1 - Faça uma previsão, usando o algoritmo de LogisticRegression.


In [0]:
#Pegar as palavras importantes.
twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...'
]

# Armazena as palavras em um dicionário que mapeia as palavras a seus índices.
count = CountVectorizer()
bag = count.fit_transform(twits)

count.vocabulary_
bag.toarray()

# Relevância das palavras
# É possível "ranquear" as palavras, por exemplo, "is" tem menos relevância que "happy".
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

tfidf.fit_transform(bag).toarray()

##Limpando palavras que não são importantes

vocab = Counter()
for twit in train.SentimentText:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)
output_notebook()

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)
nltk.download('stopwords')

stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

## finalmente treinando o algoritmo de Logistic Regression
# separar as duas databases
X = train['SentimentText']
y = train['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)


print('Melhor parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Melhor Acurácia: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_
print('Acurácia em teste: %.3f' % clf.score(X_test, y_test))
pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

twits = [
    "This is really bad, I don't like it at all",
    "I love this!",
    ":)",
    "I'm sad... :("
]

preds = clf.predict(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> {preds[i]}')
    

    
##### OBS:
## Esse tutorial eu peguei no canal do Siraj Raval (Muito bom!)
## https://www.youtube.com/watch?v=H6ii7NFdDeg

## Não consegui fazer funcionar a tempo de entregar,
## apesar de não parecer difícil exigiu um conhecimento mais aprofundado de python.
## Meu objetivo é finalizar esse projeto, independente da matéria.

# Questão 5 - valor (1,5)


Usando o método de avaliação de acurácia (Treinar e testar na base de dados inteira/Train test entire model), Calcular a acurácia de cada um dos 3 métodos abaixo.

4.1 - Acurácia usando o algoritmo de KNN (com 1 vizinho, k=1)


4.2 - Acurácia usando o algoritmo de KNN (com 5 vizinho, k=5)


4.3 - Acurácia usando o algoritmo de LogisticRegression


4.4 - De acordo com seus resultados anteriores, qual dos 3 métodos é mais eficiente?


In [0]:
*** Envie o link do seu Github com essa resposta no Classroom
